# SiamMAE

In [1]:
from mae.models_mae import  mae_vit_base_patch16
import torch

/home/eele/miniconda3/envs/pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# I want the Kinetics-400 dataset but I only want to download 5% of it

from dataset_loader.dataset import get_kinetics_dataset

/home/eele/miniconda3/envs/pytorch/lib/python3.11/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


In [3]:
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    
print("Using device =", device)

Using device = cuda


In [4]:
data = get_kinetics_dataset("./dataset/")

Loading dataset from ./dataset/


In [5]:
print(data.num_videos)

903


In [6]:
BATCH_SIZE = 1

dloader = torch.utils.data.DataLoader(
        data,
        batch_size=BATCH_SIZE,
    )